# Lab Instructions

Find a dataset that interests you. I'd recommend starting on [Kaggle](https://www.kaggle.com/). Read through all of the material about the dataset and download a .CSV file.

1. Write a short summary of the data.  Where did it come from?  How was it collected?  What are the features in the data?  Why is this dataset interesting to you?  

2. Identify 5 interesting questions about your data that you can answer using Pandas methods.  

3. Answer those questions!  You may use any method you want (including LLMs) to help you write your code; however, you should use Pandas to find the answers.  LLMs will not always write code in this way without specific instruction.  

4. Write the answer to your question in a text box underneath the code you used to calculate the answer.



In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('amazon_prime_titles.csv')

# Display basic information about the dataset
print("=== DATASET INFORMATION ===")
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nData types:")
print(df.dtypes)
print(f"\nMissing values:")
print(df.isnull().sum())

# Summary statistics for numerical columns
print(f"\n=== SUMMARY STATISTICS ===")
print(df.describe(include='all'))

=== DATASET INFORMATION ===
Dataset shape: (9668, 12)

Columns: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

First few rows:
  show_id   type                 title        director  \
0      s1  Movie   The Grand Seduction    Don McKellar   
1      s2  Movie  Take Care Good Night    Girish Joshi   
2      s3  Movie  Secrets of Deception     Josh Webber   
3      s4  Movie    Pink: Staying True  Sonia Anderson   
4      s5  Movie         Monster Maker    Giles Foster   

                                                cast         country  \
0     Brendan Gleeson, Taylor Kitsch, Gordon Pinsent          Canada   
1   Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar           India   
2  Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...   United States   
3  Interviews with: Pink, Adele, Beyoncé, Britney...   United States   
4  Harry Dean Stanton, Kieran O'Brien, George Cos...  United Kingdom  

The source of this dataset is: https://www.kaggle.com/datasets/shivamb/amazon-prime-movies-and-tv-shows?resource=download

The collection method is: from Wikipedia, FlixDatabase using Regular API calls

The features are: 
type (Movie or Tv Show)
title (Title of Movie or Show)
director (Director of Movie or Show)
cast (Cast of Movie / Show)
country (Country of Production)
date_added (Date added on Prime)
release_year (Release Year of the movie or show)
rating (Rating of the movie or show)
duration (Duration of the movie or show)

This dataset is interesting to me because I watch a lot of streaming content from amazon and it is cool to see the content from a high level data analytics perspective.

In [3]:
# Question 1: What is the distribution of Movies vs TV Shows on Amazon Prime?

# Q1: Distribution of Movies vs TV Shows
content_distribution = df['type'].value_counts()
content_percentage = df['type'].value_counts(normalize=True) * 100

print("=== QUESTION 1: MOVIES VS TV SHOWS ===")
print("Count by type:")
print(content_distribution)
print("\nPercentage distribution:")
print(content_percentage.round(2))

# Visual representation
print("\nAmazon Prime has:")
for content_type, count in content_distribution.items():
    percentage = content_percentage[content_type]
    print(f"  • {count:,} {content_type}s ({percentage:.1f}%)")

=== QUESTION 1: MOVIES VS TV SHOWS ===
Count by type:
type
Movie      7814
TV Show    1854
Name: count, dtype: int64

Percentage distribution:
type
Movie      80.82
TV Show    19.18
Name: proportion, dtype: float64

Amazon Prime has:
  • 7,814 Movies (80.8%)
  • 1,854 TV Shows (19.2%)


Amazon Prime has:
  • 7,814 Movies (80.8%)
  • 1,854 TV Shows (19.2%)

In [5]:
# Question 2: Which countries produce the most content on Amazon Prime?

# Q2: Top content-producing countries
# Clean the country column (some entries have multiple countries)
df['country_clean'] = df['country'].astype(str).str.split(', ').str[0]
df['country_clean'] = df['country_clean'].replace('nan', 'Unknown')

top_countries = df['country_clean'].value_counts().head(10)

print("\n=== QUESTION 2: TOP CONTENT-PRODUCING COUNTRIES ===")
print("Top 10 countries by number of titles:")
for i, (country, count) in enumerate(top_countries.items(), 1):
    print(f"{i}. {country}: {count:,} titles")

# Calculate percentage of total
total_titles = len(df)
print(f"\nThe top country produces {top_countries.iloc[0] / total_titles * 100:.1f}% of all content")


=== QUESTION 2: TOP CONTENT-PRODUCING COUNTRIES ===
Top 10 countries by number of titles:
1. Unknown: 8,996 titles
2. United States: 292 titles
3. India: 238 titles
4. United Kingdom: 48 titles
5. Canada: 25 titles
6. Italy: 10 titles
7. Spain: 9 titles
8. Germany: 8 titles
9. France: 6 titles
10. Australia: 6 titles

The top country produces 93.0% of all content


The most common country is "Unknown" but of the actually recorded countries the United States has the most titles at 292.

In [15]:
# Question 3 how was the movie to tv ratio changed over the years?

# Q3: Catalog composition by release year - REVISED VERSION
print("\n=== QUESTION 3: CATALOG COMPOSITION BY RELEASE YEAR ===")

# Use release_year instead of date_added
release_year_counts = df['release_year'].value_counts().sort_index()

# Remove NaN values and show recent decades
release_year_counts = release_year_counts[release_year_counts.index.notna()]

# Filter for years after 1950 to focus on more recent decades
recent_years = release_year_counts[release_year_counts.index >= 1950]

print("Number of titles released each year (since 1950):")
print("Year | Count")
print("-" * 20)

# Group by decade for better visualization
df['decade'] = (df['release_year'] // 10) * 10
decade_counts = df[df['release_year'] >= 1950]['decade'].value_counts().sort_index()

print("\nNumber of titles by decade:")
for decade, count in decade_counts.items():
    print(f"{decade}s: {count:,} titles")

# Calculate percentage by decade
print("\nPercentage of catalog by decade:")
total_recent = df[df['release_year'] >= 1950].shape[0]
for decade, count in decade_counts.sort_index(ascending=False).items():
    percentage = (count / total_recent) * 100
    print(f"{decade}s: {percentage:.1f}%")

# Find oldest and newest titles
oldest_year = df['release_year'].min()
newest_year = df['release_year'].max()
oldest_titles = df[df['release_year'] == oldest_year][['title', 'type']].head(3)
newest_titles = df[df['release_year'] == newest_year][['title', 'type']].head(3)

print(f"\nOldest release year: {int(oldest_year)}")
print(f"Newest release year: {int(newest_year)}")

print("\nTrend analysis:")
# Calculate moving average to see trends
years = release_year_counts.index.astype(int)
counts = release_year_counts.values

# Get last 30 years of data
recent_data = df[df['release_year'] >= 1990]
yearly_recent = recent_data['release_year'].value_counts().sort_index()

if len(yearly_recent) > 5:
    # Calculate 5-year moving average
    moving_avg = yearly_recent.rolling(window=5, min_periods=1).mean()
    
    print("\nRecent trends (5-year moving average):")
    for year in range(2015, 2022):
        if year in moving_avg.index:
            print(f"{year}: {moving_avg[year]:.0f} titles (avg)")

# Distribution by type over time
print("\nContent type distribution in recent years:")
for year in range(2015, 2022):
    year_data = df[df['release_year'] == year]
    if not year_data.empty:
        movies = (year_data['type'] == 'Movie').sum()
        tv_shows = (year_data['type'] == 'TV Show').sum()
        total = len(year_data)
        movie_pct = (movies / total * 100) if total > 0 else 0
        print(f"{year}: {movies} movies, {tv_shows} TV shows ({movie_pct:.0f}% movies)")


=== QUESTION 3: CATALOG COMPOSITION BY RELEASE YEAR ===
Number of titles released each year (since 1950):
Year | Count
--------------------

Number of titles by decade:
1950s: 155 titles
1960s: 139 titles
1970s: 194 titles
1980s: 299 titles
1990s: 577 titles
2000s: 1,185 titles
2010s: 4,383 titles
2020s: 2,404 titles

Percentage of catalog by decade:
2020s: 25.7%
2010s: 46.9%
2000s: 12.7%
1990s: 6.2%
1980s: 3.2%
1970s: 2.1%
1960s: 1.5%
1950s: 1.7%

Oldest release year: 1920
Newest release year: 2021

Trend analysis:

Recent trends (5-year moving average):
2015: 312 titles (avg)
2016: 366 titles (avg)
2017: 428 titles (avg)
2018: 495 titles (avg)
2019: 603 titles (avg)
2020: 719 titles (avg)
2021: 904 titles (avg)

Content type distribution in recent years:
2015: 282 movies, 96 TV shows (75% movies)
2016: 402 movies, 119 TV shows (77% movies)
2017: 404 movies, 158 TV shows (72% movies)
2018: 438 movies, 185 TV shows (70% movies)
2019: 730 movies, 199 TV shows (79% movies)
2020: 736 mov

Content type distribution in recent years:

2015: 282 movies, 96 TV shows (75% movies)
2016: 402 movies, 119 TV shows (77% movies)
2017: 404 movies, 158 TV shows (72% movies)
2018: 438 movies, 185 TV shows (70% movies)
2019: 730 movies, 199 TV shows (79% movies)
2020: 736 movies, 226 TV shows (77% movies)
2021: 1139 movies, 303 TV shows (79% movies)

In [11]:
# Question 4: What are the most common content ratings?

# Q4: Most common content ratings
ratings_distribution = df['rating'].value_counts().head(10)

print("\n=== QUESTION 4: MOST COMMON CONTENT RATINGS ===")
print("Top 10 content ratings:")
for rating, count in ratings_distribution.items():
    percentage = (count / len(df)) * 100
    print(f"{rating}: {count:,} titles ({percentage:.1f}%)")

# Analyze ratings by content type
print("\nRatings by content type:")
ratings_by_type = pd.crosstab(df['rating'], df['type']).sort_values('Movie', ascending=False)
print(ratings_by_type.head())


=== QUESTION 4: MOST COMMON CONTENT RATINGS ===
Top 10 content ratings:
13+: 2,117 titles (21.9%)
16+: 1,547 titles (16.0%)
ALL: 1,268 titles (13.1%)
18+: 1,243 titles (12.9%)
R: 1,010 titles (10.4%)
PG-13: 393 titles (4.1%)
7+: 385 titles (4.0%)
PG: 253 titles (2.6%)
NR: 223 titles (2.3%)
TV-14: 208 titles (2.2%)

Ratings by content type:
type    Movie  TV Show
rating                
13+      1849      268
16+      1272      275
18+      1097      146
R        1010        0
ALL       988      280


The most common content rating is 13+ with 2,117 titles (21.9%)

In [13]:
# Question 5: Who are the most frequent directors and actors on the platform?

# Q5: Most frequent directors and actors
print("\n=== QUESTION 5: MOST FREQUENT DIRECTORS AND ACTORS ===")

# Directors (handling multiple directors per title)
director_counts = df['director'].str.split(', ').explode().value_counts().head(10)
print("Top 10 directors by number of titles:")
for i, (director, count) in enumerate(director_counts.items(), 1):
    print(f"{i}. {director}: {count} titles")

# Actors (handling multiple actors per title)
actor_counts = df['cast'].str.split(', ').explode().value_counts().head(10)
print("\nTop 10 actors by number of titles:")
for i, (actor, count) in enumerate(actor_counts.items(), 1):
    print(f"{i}. {actor}: {count} titles")

# Calculate diversity metrics
unique_directors = df['director'].str.split(', ').explode().nunique()
unique_actors = df['cast'].str.split(', ').explode().nunique()
print(f"\nDiversity metrics:")
print(f"Unique directors: {unique_directors:,}")
print(f"Unique actors: {unique_actors:,}")


=== QUESTION 5: MOST FREQUENT DIRECTORS AND ACTORS ===
Top 10 directors by number of titles:
1. Mark Knight: 114 titles
2. Cannis Holder: 62 titles
3. Moonbug Entertainment: 37 titles
4. Jay Chapman: 34 titles
5. Arthur van Merwijk: 30 titles
6. Manny Rodriguez: 23 titles
7. John English: 20 titles
8. Brian Volk-Weiss: 19 titles
9. D.J. Viola: 19 titles
10. 1: 16 titles

Top 10 actors by number of titles:
1. Maggie Binkley: 56 titles
2. 1: 35 titles
3. Gene Autry: 32 titles
4. Nassar: 30 titles
5. Champion: 29 titles
6. Anne-Marie Newland: 25 titles
7. Prakash Raj: 24 titles
8. John Wayne: 23 titles
9. Roy Rogers: 23 titles
10. Danny Trejo: 22 titles

Diversity metrics:
Unique directors: 6,220
Unique actors: 31,917


Answer 5: MOST FREQUENT DIRECTORS AND ACTORS
Top 10 directors by number of titles:
1. Mark Knight: 114 titles
2. Cannis Holder: 62 titles
3. Moonbug Entertainment: 37 titles
4. Jay Chapman: 34 titles
5. Arthur van Merwijk: 30 titles
6. Manny Rodriguez: 23 titles
7. John English: 20 titles
8. Brian Volk-Weiss: 19 titles
9. D.J. Viola: 19 titles
10. 1: 16 titles

Top 10 actors by number of titles:
1. Maggie Binkley: 56 titles
2. 1: 35 titles
3. Gene Autry: 32 titles
4. Nassar: 30 titles
5. Champion: 29 titles
6. Anne-Marie Newland: 25 titles
7. Prakash Raj: 24 titles
8. John Wayne: 23 titles
9. Roy Rogers: 23 titles
10. Danny Trejo: 22 titles

Diversity metrics:
Unique directors: 6,220
Unique actors: 31,917